# VisCPU: data pre-processing

Metrics to calculate:
- IPC (Instructions Per Cycle): how much CPU cycles instructions require to execute?
    - How to calculate? Instructions divided by CPU cycles;
    - How to interpret? In general, values higher than one are good;
    - https://stackoverflow.com/questions/51438407/how-to-correctly-measure-ipc-instructions-per-cycle-with-perf
    
- https://stackoverflow.com/questions/22165299/what-are-stalled-cycles-frontend-and-stalled-cycles-backend-in-perf-stat-resul

- How well cache is working?
    - How to calculate? Cache misses divided by instructions;

## Dependencies and imports

Install and import required packages.

In [ ]:
!pip install pandas
!pip install psutil
!conda install -c plotly plotly-orca -y

In [1]:
import json
import pandas as pd
import numpy as np
from viscpu import utils, perf_record

%load_ext autoreload
%autoreload 2

## perf record

### Parse datasets

In [96]:
dataset_1 = "../applications/simple-ff-test/data/perf-record-1.txt"
dataset_1_output = "../applications/simple-ff-test/data/perf-record-1.csv"
dataset_2 = "../applications/simple-ff-test/data/perf-record-2.txt"
dataset_2_output = "../applications/simple-ff-test/data/perf-record-2.csv"

# perf_record.parse_record_dataset(dataset_1, dataset_1_output)
# perf_record.parse_record_dataset(dataset_2, dataset_2_output)

df_1 = pd.read_csv(dataset_1_output)
df_2 = pd.read_csv(dataset_2_output)

In [97]:
df_1["time"] = df_1["time"] - df_1["time"].min()
df_1["time_second"] = df_1["time"].apply(lambda x: int(x))
df_1 = df_1.groupby(["time_second", "tid", "event", "cpu"]).agg({"counter": "sum", "time": "mean", "stack": perf_record.agg_stack}).reset_index()
# print(df_1.tail())

df_2["time"] = df_2["time"] - df_2["time"].min()
df_2["time_second"] = df_2["time"].apply(lambda x: int(x))
df_2 = df_2.groupby(["time_second", "tid", "event", "cpu"]).agg({"counter": "sum", "time": "mean", "stack": perf_record.agg_stack}).reset_index()
# print(df_2.tail())

### Process datasets

In [98]:
events = df_1["event"].unique().tolist()
cpu_setup = np.zeros([3, 4], dtype=int).tolist()
cpu_labels = perf_record.cpu_labels(cpu_setup, 4)

output = {
    "events": events,
    "cpu_setup": cpu_setup,
    "cpu_labels": cpu_labels,
    "threads-dataset-1": df_1["tid"].unique().tolist(),
    "threads-dataset-2": df_2["tid"].unique().tolist(),
    "dataset-1": {"raw": {}, "aggregated": {}},
    "dataset-2": {"raw": {}, "aggregated": {}},
    "comparison-1-2": {},
    "comparison-2-1": {}
}

Load data from each captured event and write in `output`:

In [99]:
for event in events:
    if event not in output["dataset-1"]["raw"]:
        output["dataset-1"]["raw"][event] = {}
        output["dataset-2"]["raw"][event] = {}
        
    times, captures = perf_record.get_event_data(df_1, cpu_setup, event)
    output["dataset-1"]["raw"][event] = {"captures": captures}

    times, captures = perf_record.get_event_data(df_2, cpu_setup, event)
    output["dataset-2"]["raw"][event] = {"captures": captures}

Aggregate time series of events. This will allow to compare the overall performance of the experiments.

In [100]:
for event in events:
    df_1_aggr = df_1[(df_1["event"] == event)].groupby(["cpu"], as_index=False)["counter"]
    df_2_aggr = df_2[(df_2["event"] == event)].groupby(["cpu"], as_index=False)["counter"]

    a = perf_record.transform_cpu_data(df_1_aggr.mean(), cpu_setup)
    a_sum = np.sum(a)
    b = perf_record.transform_cpu_data(df_2_aggr.mean(), cpu_setup)
    b_sum = np.sum(b)

    output["dataset-1"]["aggregated"][event] = {
        "mean": a,
        "mean_relative": a if a_sum <= 0 else ((np.array(a) / a_sum) * 100).tolist()
    }
    output["dataset-2"]["aggregated"][event] = {
        "mean": b,
        "mean_relative": b if b_sum <= 0 else ((np.array(b) / b_sum) * 100).tolist()
    }

    output["comparison-1-2"][event] = {
        "mean": (np.array(a) - np.array(b)).tolist(),
        "mean_relative": b if b_sum <= 0 else ((np.array(a) / a_sum - np.array(b) / b_sum) * 100).tolist(),
        "mean_value": 0 if np.mean(a) <= 0 else ((np.mean(b) / np.mean(a)) - 1) * 100
    }
    output["comparison-2-1"][event] = {
        "mean": (np.array(b) - np.array(a)).tolist(),
        "mean_relative": a if a_sum <= 0 else ((np.array(b) / b_sum - np.array(a) / a_sum) * 100).tolist(),
        "mean_value": 0 if np.mean(b) <= 0 else ((np.mean(a) / np.mean(b)) - 1) * 100
    }

### Output

Write `output` to a JSON file:

In [101]:
!mkdir data

with open("data/simple-ff-test-1-2.json", "w") as f:
    json.dump(output, f)

mkdir: cannot create directory ‘data’: File exists
